<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/2.%20coxph_find_best_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [30]:
pip install pycox

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [32]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [33]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [34]:
dataset_val = dataset.sample(frac=0.2)
dataset_train = dataset.drop(dataset_val.index)
dataset_test = dataset_train.sample(frac=0.2)
dataset_train = dataset_train.drop(dataset_test.index)

- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [35]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [36]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [37]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [38]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [39]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2,3,4]
number_nodes = [32, 64, 128, 256, 512, 1024, 2048]
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0]
brier_scores = []

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)
      batch_size = 639

      if k == 0:
        lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
        model.optimizer.set_lr(lrfinder.get_best_lr())
      else:
        model.optimizer.set_lr(k)
      
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)
      
      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)

      if k == 0:
        brier_scores.append([score, i, j, lrfinder.get_best_lr(), treat_hr[:10]])
      else:
        brier_scores.append([score, i, j, k, treat_hr[:10]])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
0:	[0s / 0s],		train_loss: 5.6326,	val_loss: 4.2812
1:	[0s / 0s],		train_loss: 5.6270,	val_loss: 4.2814
2:	[0s / 0s],		train_loss: 5.6090,	val_loss: 4.2816
3:	[0s / 0s],		train_loss: 5.6052,	val_loss: 4.2818
4:	[0s / 0s],		train_loss: 5.5874,	val_loss: 4.2821
5:	[0s / 0s],		train_loss: 5.5887,	val_loss: 4.2824
6:	[0s / 0s],		train_loss: 5.5788,	val_loss: 4.2827
7:	[0s / 0s],		train_loss: 5.5776,	val_loss: 4.2831
8:	[0s / 0s],		train_loss: 5.5634,	val_loss: 4.2834
9:	[0s / 0s],		train_loss: 5.5376,	val_loss: 4.2839
10:	[0s / 0s],		train_loss: 5.5442,	val_loss: 4.2843
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
0:	[0s / 0s],		train_loss: 5.5774,	val_loss: 4.2854
1:	[0s / 0s],		train_loss: 5.4934,	val_loss: 4.2839
2:	[0s / 0s],		train_loss: 5.4250,	val_loss: 4.2824
3:	[0s / 0s],		train_loss: 5.3810,	val_loss: 4.2807
4:	[0s / 0s],		train_loss: 5.3233,	val_loss: 4.2790
5:	[0s / 0s],		train_loss: 5.3012,	val_

/usr/local/lib/python3.7/dist-packages/pycox/models/cox.py:237: RuntimeWarning: overflow encountered in exp
  .assign(expg=np.exp(self.predict(input, batch_size, True, eval_, num_workers=num_workers)))


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
0:	[0s / 0s],		train_loss: 5.6618,	val_loss: 4.2681
1:	[0s / 0s],		train_loss: 6.4053,	val_loss: 4.2768
2:	[0s / 0s],		train_loss: 8.9510,	val_loss: 4.2811
3:	[0s / 1s],		train_loss: 8.0917,	val_loss: 4.2730
4:	[0s / 1s],		train_loss: 6.1158,	val_loss: 4.2623
5:	[0s / 1s],		train_loss: 5.4639,	val_loss: 4.2562
6:	[0s / 2s],		train_loss: 5.2053,	val_loss: 4.2535
7:	[0s / 2s],		train_loss: 5.1269,	val_loss: 4.2507
8:	[0s / 2s],		train_loss: 4.9162,	val_loss: 4.2438
9:	[0s / 3s],		train_loss: 4.7610,	val_loss: 4.2323
10:	[0s / 3s],		train_loss: 4.6748,	val_loss: 4.2185
11:	[0s / 3s],		train_loss: 4.6425,	val_loss: 4.2033
12:	[0s / 3s],		train_loss: 4.6067,	val_loss: 4.1862
13:	[0s / 4s],		train_loss: 4.4640,	val_loss: 4.1675
14:	[0s / 4s],		train_loss: 4.3902,	val_loss: 4.1494
15:	[0s / 4s],		train_loss: 4.2704,	val_loss: 4.1363
16:	[0s / 5s],		train_loss: 4.1842,	val_loss: 4.1312
17:	[0s / 5s],		train_loss: 4.0831,	val_loss:

/usr/local/lib/python3.7/dist-packages/pycox/models/cox.py:237: RuntimeWarning: overflow encountered in exp
  .assign(expg=np.exp(self.predict(input, batch_size, True, eval_, num_workers=num_workers)))
/usr/local/lib/python3.7/dist-packages/pycox/models/cox.py:257: RuntimeWarning: overflow encountered in exp
  expg = np.exp(self.predict(input, batch_size, True, eval_, num_workers=num_workers)).reshape(1, -1)


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
0:	[0s / 0s],		train_loss: 5.6022,	val_loss: 4.2762
1:	[0s / 0s],		train_loss: 5.5121,	val_loss: 4.2764
2:	[0s / 1s],		train_loss: 5.3065,	val_loss: 4.2750
3:	[0s / 1s],		train_loss: 5.1414,	val_loss: 4.2732
4:	[0s / 2s],		train_loss: 4.9246,	val_loss: 4.2709
5:	[0s / 2s],		train_loss: 4.7021,	val_loss: 4.2675
6:	[0s / 2s],		train_loss: 4.6416,	val_loss: 4.2630
7:	[0s / 3s],		train_loss: 4.4308,	val_loss: 4.2574
8:	[0s / 3s],		train_loss: 4.4339,	val_loss: 4.2520
9:	[0s / 4s],		train_loss: 4.2968,	val_loss: 4.2422
10:	[0s / 4s],		train_loss: 4.2225,	val_loss: 4.2281
11:	[0s / 5s],		train_loss: 4.1616,	val_loss: 4.2112
12:	[0s / 5s],		train_loss: 4.1104,	val_loss: 4.1949
13:	[0s / 5s],		train_loss: 4.0273,	val_loss: 4.1810
14:	[0s / 6s],		train_loss: 3.9576,	val_loss: 4.1706
15:	[0s / 6s],		train_loss: 3.9843,	val_loss: 4.1666
16:	[0s / 7s],		train_loss: 3.8934,	val_loss: 4.1665
17:	[0s / 7s],		train_loss: 4.0013,	val_loss:

/usr/local/lib/python3.7/dist-packages/pycox/models/cox.py:237: RuntimeWarning: overflow encountered in exp
  .assign(expg=np.exp(self.predict(input, batch_size, True, eval_, num_workers=num_workers)))
/usr/local/lib/python3.7/dist-packages/pycox/models/cox.py:257: RuntimeWarning: overflow encountered in exp
  expg = np.exp(self.predict(input, batch_size, True, eval_, num_workers=num_workers)).reshape(1, -1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
0:	[1s / 1s],		train_loss: 5.6899,	val_loss: 4.2762
1:	[1s / 2s],		train_loss: 5.7321,	val_loss: 4.2766
2:	[1s / 3s],		train_loss: 6.1326,	val_loss: 4.2745
3:	[1s / 4s],		train_loss: 6.5103,	val_loss: 4.2718
4:	[1s / 5s],		train_loss: 5.9244,	val_loss: 4.2691
5:	[1s / 6s],		train_loss: 5.6352,	val_loss: 4.2653
6:	[1s / 8s],		train_loss: 5.4293,	val_loss: 4.2624
7:	[1s / 9s],		train_loss: 5.3082,	val_loss: 4.2573
8:	[1s / 10s],		train_loss: 5.0089,	val_loss: 4.2528
9:	[1s / 11s],		train_loss: 4.9460,	val_loss: 4.2469
10:	[1s / 12s],		train_loss: 4.8723,	val_loss: 4.2350
11:	[1s / 13s],		train_loss: 4.6730,	val_loss: 4.2228
12:	[1s / 15s],		train_loss: 4.5596,	val_loss: 4.2111
13:	[1s / 16s],		train_loss: 4.5137,	val_loss: 4.2017
14:	[1s / 17s],		train_loss: 4.3707,	val_loss: 4.1917
15:	[1s / 18s],		train_loss: 4.3115,	val_loss: 4.1842
16:	[1s / 19s],		train_loss: 4.4624,	val_loss: 4.1777
17:	[1s / 21s],		train_loss: 4.1617,

- brier_score가 가장 작은 것부터 정렬


In [40]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

- brier_score가 가장 좋은 케이스 출력

In [41]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.05486206931620891
Hidden Layers : 2
Number of Nodes : 2048
Learning Rate : 0.0012328467394420717
Selection : ['G186', 'G204', 'G149', 'G211', 'G89', 'G137', 'G33', 'G247', 'G28', 'G101']


NAN 값을 기준으로 정렬이 끊어진 문제 발견

In [42]:
print("brier_score, hidden layer, number of nodes, learning rate")
for i in brier_scores:
  print(i[:-1])

brier_score, hidden layer, number of nodes, learning rate
[0.05486206931620891, 2, 2048, 0.0012328467394420717]
[0.05599108009029401, 2, 2048, 0.001]
[0.05649850279405245, 2, 2048, 0.0001]
[0.056879551921934245, 2, 1024, 0.0037649358067924866]
[0.05745079570515864, 1, 1024, 0.01]
[0.05767288154328691, 1, 2048, 0.002154434690031894]
[0.05786886973884575, 2, 512, 0.0001]
[0.0581968006573943, 2, 512, 0.001]
[0.05867504098944437, 2, 128, 0.01]
[0.05874800988255423, 2, 256, 0.001]
[0.058876607126548235, 1, 512, 0.01]
[0.05889672388425608, 1, 1024, 0.002595024211399749]
[0.05896213132705461, 2, 512, 0.006579332246575716]
[0.059118759338349945, 1, 2048, 0.001]
[0.0592954023921273, 2, 256, 0.004534878508128605]
[0.059295568141000135, 2, 256, 0.01]
[0.05933670459466444, 1, 256, 0.01]
[0.05980068592028103, 1, 512, 0.001]
[0.059887059455372475, 1, 64, 0.01]
[0.06013645976547433, 1, 1024, 0.001]
[0.060268235336821345, 2, 1024, 0.0001]
[0.06027560314360651, 2, 1024, 0.001]
[0.060337261768657766, 2,